# Code Llama-Instruct 7b Evaluation - BLEU Score
The BLEU (Bilingual Evaluation Understudy) score is a metric commonly used to evaluate the quality of machine-generated text,
particularly in the context of natural language generation tasks like code summarization. In this notebook, we use the BLEU
score to evaluate the quality of code generated by the CodeLlama - Instruct 7b model in response to natural language prompts from the
CoNaLa dataset. <br>
Here's how the evaluation process works:
1. We load the CoNaLa dataset, which contains natural language prompts (intents) and corresponding code snippets.
2. We load the pre-trained CodeLlama - Instruct 7b model pipeline, which is specifically fine-tuned for code generation tasks following natural language instructions.
3. We iterate through the testing set of the dataset, generating code snippets based on the provided intents using the model.
4. For each generated code snippet, we calculate its BLEU score against the reference code snippet from the dataset.
5. Finally, we compute the average BLEU score across all generated code snippets to evaluate the overall performance of the model.



In [ ]:
import transformers
from datasets import load_dataset
from nltk.translate.bleu_score import sentence_bleu

In [ ]:
# Load CoNaLa dataset
dataset = load_dataset("neulab/conala")

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Display dataset information
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['question_id', 'intent', 'rewritten_intent', 'snippet'],
        num_rows: 2379
    })
    test: Dataset({
        features: ['question_id', 'intent', 'rewritten_intent', 'snippet'],
        num_rows: 500
    })
})


In [ ]:
import torch

In [ ]:
# Create pipeline for Code Llama
model = transformers.pipeline(
    "text-generation",
    model="codellama/CodeLlama-7b-Instruct-hf",
    torch_dtype=torch.float32,
    device_map="auto"  # Use GPU if available
)

2024-03-18 11:14:54.716872: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-18 11:14:54.716928: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-18 11:14:54.720586: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Iterate through testing set to generate code and calculate BLEU score
bleu_scores = []
for example in dataset["test"]:

    # Extract the natural language prompt (intent)
    prompt = example["intent"]

    # Reference code for BLEU score calculation
    reference_code = example["snippet"]  # Expected code snippet

    # Generate code using the model pipeline
    output = model(prompt, max_length=50)
    generated_code = output[0].get("generated_text")  # Access generated code

    # Calculate BLEU score
    bleu = sentence_bleu([reference_code.split()], generated_code.split())
    bleu_scores.append(bleu)

In [ ]:
# Print average BLEU score
print(f"Average BLEU score: {sum(bleu_scores) / len(bleu_scores)}")

Average BLEU score: 0.0952474061356062


The BLEU score ranges from 0 to 1, with higher scores indicating better quality and similarity between the generated and
reference code snippets. However, it's important to note that the BLEU score has limitations, such as being sensitive to
lexical similarity and not capturing semantic equivalence perfectly. Therefore, while BLEU score provides a quantitative
measure of performance, it should be interpreted alongside qualitative assessments and domain-specific considerations.
